# Inspecteer Excel-bestanden in raw_batch_geoloket
Deze notebook toont de eerste 5 regels van elke tabblad in elk bestand in de map `raw_batch_geoloket`.

In [1]:
import os
from pathlib import Path
from openpyxl import load_workbook

# Use correct relative path from the notebooks folder
folder = Path('../input_files/raw_batch_geoloket').resolve()
if not folder.exists():
    raise FileNotFoundError(f"Map niet gevonden: {folder}")

files = [f for f in os.listdir(folder) if f.endswith('.xlsx')]
for fname in files:
    print(f'Bestand: {fname}')
    wb = load_workbook(folder / fname, data_only=True, read_only=True)
    for sheet in wb.sheetnames:
        print(f'  Tabblad: {sheet}')
        ws = wb[sheet]
        for i, row in enumerate(ws.iter_rows(values_only=True), 1):
            print(row)
            if i >= 5:
                break
        print()

Bestand: 83034-1.xlsx
  Tabblad: 83034-1 HB001PB01
('83034-1 HB001PB01', 'ELLI_D_14511368_airtemp', 'ELLI_D_14511368', None)
('Timestamp', 'Lucht Temperatuur (ºC)', 'Waterdrukhoogte (cm H₂O)', 'Water Temperatuur (ºC)')
(datetime.datetime(2025, 6, 24, 22, 0), 20.32, 159.82, 12.67)
(datetime.datetime(2025, 6, 24, 23, 0), 19.83, 161.11, 12.67)
(datetime.datetime(2025, 6, 25, 0, 0), 19.35, 159.56, 12.67)

  Tabblad: 83034-1 HB005PB01
('83034-1 HB005PB01', 'ELLI_D_22021509_airtemp', 'ELLI_D_22021509', None)
('Timestamp', 'Lucht Temperatuur (ºC)', 'Waterdrukhoogte (cm H₂O)', 'Water Temperatuur (ºC)')
(datetime.datetime(2025, 6, 6, 5, 0), 15.59, 190.69, 11.27)
(datetime.datetime(2025, 6, 6, 6, 0), 15.44, 190.79, 11.23)
(datetime.datetime(2025, 6, 6, 7, 0), 15.33, 190.9, 11.27)

  Tabblad: 83034-1 HB007PB01
('83034-1 HB007PB01', 'ELLI_D_22021526_airtemp', 'ELLI_D_22021526', None)
('Timestamp', 'Lucht Temperatuur (ºC)', 'Waterdrukhoogte (cm H₂O)', 'Water Temperatuur (ºC)')
(datetime.datetime(20

In [2]:
import pandas as pd
import re
from pathlib import Path

# === CONFIG ===
# Folder with input files
input_folder = Path('../input_files/raw_batch_geoloket').resolve()
output_format = 'csv'  # 'csv' of 'xlsx'
output_dir = Path('../output_files/output_sheets').resolve()
output_dir.mkdir(exist_ok=True)

# === SCRIPT ===
all_summaries = []
files = [f for f in input_folder.iterdir() if f.suffix == '.xlsx']
for input_file in files:
    try:
        xls = pd.ExcelFile(input_file)
    except Exception as e:
        print(f'Fout bij openen van bestand {input_file}: {e}')
        continue

    summary = []
    for sheet in xls.sheet_names:
        # Read first two rows: row 0 for name, row 1 for headers
        try:
            preview = pd.read_excel(xls, sheet_name=sheet, nrows=2, header=None)
        except Exception as e:
            print(f'Fout bij lezen van eerste rijen van sheet {sheet} in {input_file.name}: {e}')
            continue
        if preview.empty or len(preview) < 2:
            print(f'{input_file.name} - Sheet {sheet}: niet genoeg rijen voor naam en headers!')
            continue
        out_name = str(preview.iloc[0,0]).strip()  # Use first cell of first row as output name
        # Now read the actual data, using row 2 (index 1) as header
        try:
            df = pd.read_excel(xls, sheet_name=sheet, header=1)
        except Exception as e:
            print(f'Fout bij lezen van sheet {sheet} in {input_file.name}: {e}')
            continue

        # Zoek kolommen
        ts_col = next((col for col in df.columns if 'Timestamp' in col), None)
        wn_col = next((col for col in df.columns if 'Waterniveau (m NAP)' in col), None)
        if not ts_col or not wn_col:
            print(f'{input_file.name} - Sheet {sheet}: vereiste kolommen niet gevonden!')
            continue

        df_out = df[[ts_col, wn_col]].copy()
        df_out = df_out.rename(columns={wn_col: 'head'})
        df_out[ts_col] = pd.to_datetime(df_out[ts_col], errors='coerce')

        out_path = output_dir / f'{out_name}.{output_format}'
        if output_format == 'csv':
            df_out.to_csv(out_path, index=False)
        elif output_format == 'xlsx':
            df_out.to_excel(out_path, index=False)
        else:
            print(f'Onbekend output-formaat: {output_format}')
            continue

        summary.append((out_name, len(df_out)))

    if summary:
        print(f'Samenvatting voor {input_file.name}:')
        for name, rows in summary:
            print(f'{name}: {rows} rijen opgeslagen.')
        all_summaries.append((input_file.name, summary))
    else:
        print(f'Geen geschikte sheets gevonden of opgeslagen in {input_file.name}.')

83034-1.xlsx - Sheet 83034-1 HB001PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 HB005PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 HB007PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 HB010PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 HB011PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 HB012PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 HB013PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 HB016PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 MB004PB02: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 MB014PB01: vereiste kolommen niet gevonden!
83034-1.xlsx - Sheet 83034-1 MB014PB02: vereiste kolommen niet gevonden!
Samenvatting voor 83034-1.xlsx:
83034-1 HB015PB01: 3 rijen opgeslagen.
Samenvatting voor 86349-1.xlsx:
86349-1 HB002PB01: 7777 rijen opgeslagen.
86349-1 HB007PB01: 2869 rijen opgeslagen.
86349-1 HB